In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
import argparse
import threading

In [ ]:
def get_subdirectories(BASE_URL):
    res = requests.get(BASE_URL)
    soup = BeautifulSoup(res.text, 'html.parser')

    rows = soup.findAll('tr')

    years = []
    for row in rows[3:-2]:
        row = row.find('a')
        years.append(row.attrs['href'])

    return years

In [ ]:
def get_url_files(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    rows = soup.findAll('tr')
    
    urls = list()

    for row in rows[3:-1]:
        row  = row.find('a')
        try:
            link = row.attrs['href']
            urls.append(os.path.join(url, link))
        except:
            pass
    return urls

In [ ]:
def download_file(url_file,file_out):
    res = requests.get(url_file, stream=True)
    total_size = int(res.headers['content-length'])

    with open(file_out, 'wb') as handle:
        for data in tqdm(iterable=res.iter_content(chunk_size=1024), total=total_size/1024, unit='KB'):
            handle.write(data)
    # print("Done!")

In [ ]:
product = 'rgpf'

In [ ]:
BASE_URL = 'http://atmos.tamucc.edu/trmm/data/gpm/level_2/'+product+'/'
list_files = get_subdirectories(BASE_URL)
print(list_files)

In [ ]:
for f in list_files:
    DIR_BASE = '/mnt/Data/Data/PF/'+product+'/'
    File_out = DIR_BASE + f

    if not os.path.exists(DIR_BASE):
        os.makedirs(DIR_BASE)

    url = BASE_URL + f

    if not os.path.exists(File_out):
        print(f"   Downloading: {f}")
        download_file(url,File_out)
    else:
        print("   File Exists")

    print()
